In [4]:
import pandas as pd
import hvplot.pandas
import requests
import json
import os
import csv
from api_keys import geoapify_key

In [85]:
csvpath = os.path.join('Resources', 'zillowzipcodedata.csv')
with open(csvpath) as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    next(csvreader)
    zillowdf = pd.DataFrame(csvreader)
zillowdf = zillowdf.loc[(zillowdf[6] == 'Philadelphia')&(zillowdf[5] == 'PA'),(2,290)]
zillowdf = zillowdf.rename(columns={2:'zip_code',290:'ZHVI'})
zillowdf = zillowdf.set_index('zip_code')
zillowdf

,ZHVI
zip_code,
19120,157079.99056692256
19124,155755.513047764
19111,279611.3807600301
19143,127886.75033201296
19149,220640.6397306033
19134,102127.55170587571
19140,80905.3843383473
19104,215241.11710614603
19148,251917.63273992765


In [50]:
philly_crime_url = 'https://phl.carto.com/api/v2/sql?q=SELECT * FROM arrests_zipcode'
philly_crime = requests.get(philly_crime_url).json()
philly_crime = pd.DataFrame(philly_crime['rows'])
philly_crime = philly_crime.loc[philly_crime['zip_code'] != 'Unknown',:]
# philly_crime = philly_crime.sample(100000)
philly_crime

,cartodb_id,the_geom,the_geom_webmercator,offense_category,day,zip_code,defendant_race,count,objectid
0,1,None,None,Aggravated Assault,2011-01-13T05:00:00Z,19121,Black,2,26227634
1,2,None,None,Aggravated Assault,2011-01-13T05:00:00Z,19130,Black,2,26227635
2,3,None,None,Aggravated Assault,2011-01-13T05:00:00Z,19131,Black,1,26227636
3,4,None,None,Aggravated Assault,2011-01-13T05:00:00Z,19132,Black,1,26227637
4,5,None,None,Aggravated Assault,2011-01-13T05:00:00Z,19143,Black,1,26227638
...,...,...,...,...,...,...,...,...,...
376993,376994,None,None,Violation of Protection Order,2020-05-02T04:00:00Z,19149,White,1,26603104
376994,376995,None,None,Violation of Protection Order,2020-05-03T04:00:00Z,19116,White,2,26603105
376995,376996,None,None,Violation of Protection Order,2020-05-04T04:00:00Z,19119,Black,1,26603106
376996,376997,None,None,Violation of Protection Order,2020-05-05T04:00:00Z,19118,Black,1,26603107


In [107]:
philly_crime = philly_crime.loc[:,('zip_code','count')]
philly_crime_counts = pd.DataFrame(philly_crime['zip_code'].value_counts())
philly_crime_counts = philly_crime_counts.rename(columns={'zip_code':'offense_count'})

In [102]:
philly_crime_counts['lat'] = ''
philly_crime_counts['lon'] = ''
for index, row in philly_crime_counts.iterrows():
    zipcode = index
    zipcodeurl = f'https://api.geoapify.com/v1/geocode/search?text={zipcode}&lang=en&limit=10&type=postcode&filter=countrycode:us&apiKey={geoapify_key}'
    zipcodecoords = requests.get(zipcodeurl).json()
    print(f'Converting zip code {zipcode} to coordinates')
    try:
        lat = zipcodecoords['features'][0]['properties']['lat']
        lon = zipcodecoords['features'][0]['properties']['lon']
        philly_crime_counts.loc[index,'lat'] = lat
        philly_crime_counts.loc[index,'lon'] = lon
        print(lat,lon)
    except:
        print('Bad item')
        lat = ''
        lon = ''
        pass

Converting zip code 19134 to coordinates
39.992856215 -75.11130586
Converting zip code 19140 to coordinates
40.009617474 -75.146059009
Converting zip code 19124 to coordinates
40.01523299 -75.088784716
Converting zip code 19133 to coordinates
39.988751517 -75.1420438
Converting zip code 19132 to coordinates
39.993517886 -75.164945773
Converting zip code 19139 to coordinates
39.96101535 -75.221072151
Converting zip code 19143 to coordinates
39.943270181 -75.222942213
Converting zip code 19121 to coordinates
39.978775174 -75.170031374
Converting zip code 19120 to coordinates
40.028970779 -75.121499097
Converting zip code 19148 to coordinates
39.919157966 -75.157547605
Converting zip code 19104 to coordinates
39.957497248 -75.197276366
Converting zip code 19131 to coordinates
39.981008869 -75.222526451
Converting zip code 19144 to coordinates
40.034325874 -75.172083913
Converting zip code 19141 to coordinates
40.035783215 -75.144103097
Converting zip code 19145 to coordinates
39.923856337

In [103]:
philly_combined = pd.merge(philly_crime_counts,zillowdf,how='left', right_on = 'zip_code',left_index=True)
philly_combined = philly_combined.dropna()
philly_combined

,zip_code,offense_count,lat,lon,ZHVI
19134,19134,36950,39.992856,-75.111306,102127.55170587571
19140,19140,21032,40.009617,-75.146059,80905.3843383473
19124,19124,19892,40.015233,-75.088785,155755.513047764
19133,19133,18365,39.988752,-75.142044,66578.0433609949
19132,19132,14620,39.993518,-75.164946,69020.88959923598
19139,19139,14573,39.961015,-75.221072,117185.50579333893
19143,19143,13962,39.94327,-75.222942,127886.75033201296
19121,19121,11353,39.978775,-75.170031,190426.19388412157
19120,19120,11286,40.028971,-75.121499,157079.99056692256
19148,19148,10765,39.919158,-75.157548,251917.63273992765


In [181]:
philly_combined['color'] = ''
philly_combined['size'] = 0
philly_combined
for index,row in philly_combined.iterrows():
    philly_combined['size'] = philly_combined['offense_count']/10
    if (philly_combined.loc[index,'ZHVI'] < 70000.):
        philly_combined.loc[index,'color'] = 'red'
    elif (philly_combined.loc[index,'ZHVI'] >= 70000.)&(philly_combined.loc[index,'ZHVI'] < 110000.):
        philly_combined.loc[index,'color'] = 'orange'
    elif (philly_combined.loc[index,'ZHVI'] >= 110000.)&(philly_combined.loc[index,'ZHVI'] < 200000.):
        philly_combined.loc[index,'color'] = 'yellow'
    elif (philly_combined.loc[index,'ZHVI'] >= 200000.)&(philly_combined.loc[index,'ZHVI'] < 300000.):
        philly_combined.loc[index,'color'] = 'green'
    elif (philly_combined.loc[index,'ZHVI'] >= 300000.)&(philly_combined.loc[index,'ZHVI'] < 400000.):
        philly_combined.loc[index,'color'] = 'blue'
    elif (philly_combined.loc[index,'ZHVI'] > 400000):
        philly_combined.loc[index,'color'] = 'indigo'
philly_combined

,zip_code,offense_count,lat,lon,ZHVI,color,size
19134,19134,36950,39.992856,-75.111306,102127.551706,orange,3695.0
19140,19140,21032,40.009617,-75.146059,80905.384338,orange,2103.2
19124,19124,19892,40.015233,-75.088785,155755.513048,yellow,1989.2
19133,19133,18365,39.988752,-75.142044,66578.043361,red,1836.5
19132,19132,14620,39.993518,-75.164946,69020.889599,red,1462.0
19139,19139,14573,39.961015,-75.221072,117185.505793,yellow,1457.3
19143,19143,13962,39.94327,-75.222942,127886.750332,yellow,1396.2
19121,19121,11353,39.978775,-75.170031,190426.193884,yellow,1135.3
19120,19120,11286,40.028971,-75.121499,157079.990567,yellow,1128.6
19148,19148,10765,39.919158,-75.157548,251917.632740,green,1076.5


In [184]:
themap = philly_combined.hvplot.points("lon","lat",geo = True,tiles = "CartoDark",frame_width = 1000,frame_height = 600, size = 'size',color='color', hover_cols=['zip_code','offense_count','ZHVI'])

In [185]:
themap

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [lon,lat]   (color,size,zip_code,offense_count,ZHVI)